# 제주도 급속 충전소 정보 데이터 프레임 생성

## load library

In [1]:
import requests
import pandas as pd
import numpy as np
import json

from tqdm import tqdm

## 공공데이터포털의 환경부 전기차 충전소 정보 api를 사용하여 충전소 정보 추출

- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15076352

In [2]:
encoding='MKrUU4zaVCpA1ox3Kz%2FmGU6pMnKkR%2FrA4R7N3%2Fxzy7caf2e3bbNlvaYnxH1vGVQFaNTVIKPYmGU5hzHycxAnVQ%3D%3D'
decoding='MKrUU4zaVCpA1ox3Kz/mGU6pMnKkR/rA4R7N3/xzy7caf2e3bbNlvaYnxH1vGVQFaNTVIKPYmGU5hzHycxAnVQ=='

pagenum=5874

url='https://apis.data.go.kr/B552584/EvCharger/getChargerInfo?serviceKey='+decoding+'&pageNo=1&numOfRows='+str(pagenum)+'&zcode=50&dataType=JSON'

# print(url)
req = requests.get(url, verify=False)

c:\work\Miniconda\envs\tf_env\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
print('총 전기차 충전소 수:',json.loads(req.text)['totalCount'])

총 전기차 충전소 수: 5874


In [4]:
chargers=json.loads(req.text)['items']['item']

## 추출한 json 데이터를 판다스 데이터프레임으로 변경

In [5]:
from pandas.io.json import json_normalize

df=json_normalize(chargers)

C:\Users\ggyo0\AppData\Local\Temp\ipykernel_22728\1986055437.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df=json_normalize(chargers)


In [6]:
df.head(1)

,statNm,statId,chgerId,chgerType,addr,location,useTime,lat,lng,busiId,...,zscode,kind,kindDetail,parkingFree,note,limitYn,limitDetail,delYn,delDetail,trafficYn
0,남원읍 공영주차장,ME174045,01,06,제주특별자치도 서귀포시 남원읍 남원리 84-2,null,24시간 이용가능,33.2788767,126.7213139,ME,...,50130,B0,B001,N,,N,,N,,N


### data preprocessing

In [8]:
charger_type=['02','07','89']


# 89는 수소충전 타입이라 포함x
# 충전소 정보 중 급속 충전기만 추출
df_fast=df[np.logical_and(df['chgerType'] != '02',df['chgerType'] != '07')]

- addr : 주소
- statNm : 충전소 위치 명
- zscode : 지역 코드(읍면동)
- kind, kindDetail : 충전소 구분 코드(공용 등..)
- parkingFree : 주차 무료 여부
- limitYn, limitDetail : 출입 제한 여부, 출입 제한 사유
- chgerType : 충전기 타입(급속 dc...)
- useTime : 사용 가능 시간
- lat, lng : 위도, 경도

In [9]:
df_fast=df_fast[['statNm','zscode','kind','kindDetail','parkingFree','limitYn','limitDetail','chgerType','useTime','lat','lng']]
df_fast.head(1)

,statNm,zscode,kind,kindDetail,parkingFree,limitYn,limitDetail,chgerType,useTime,lat,lng
0,남원읍 공영주차장,50130,B0,B001,N,N,,06,24시간 이용가능,33.2788767,126.7213139


In [12]:
df_fast=df_fast.reset_index().drop(['index'], axis=1) # 인덱스 초기화

### 충전소 위치 명을 기준으로 그룹핑하여 급속 충전기 개수 체크

In [14]:

df_fast_count=df_fast[['statNm','zscode']].groupby('statNm').count().reset_index().rename(columns={'zscode':'count'})

### 충전소 데이터프레임의 위도, 경도 소수점 3자리까지만 표시

In [61]:
df_fast['lat']=df_fast['lat'].str[:6]
df_fast['lng']=df_fast['lng'].str[:7]
df_fast.head(3)

,statNm,zscode,kind,kindDetail,parkingFree,limitYn,limitDetail,chgerType,useTime,lat,lng
0,남원읍 공영주차장,50130,B0,B001,N,N,,06,24시간 이용가능,33.278,126.721
1,위미의례회관,50130,A0,A004,N,N,,06,24시간 이용가능,33.289,126.657
2,대정감협 인근 공영주차장,50130,B0,B001,N,N,,06,24시간 이용가능,33.222,126.252


### 중복 데이터를 제거하고 위치 명을 기준으로 df_fast_count와 merge하여 충전기 대수 표시

In [15]:
df_fast.duplicated().sum()

726

In [16]:
# 중복 데이터 정보
df_fast[df_fast.duplicated()]

,statNm,zscode,kind,kindDetail,parkingFree,limitYn,limitDetail,chgerType,useTime,lat,lng
6,중문 관광단지,50130,D0,D007,N,N,,06,24시간 이용가능,33.2462149,126.414261
7,중문 관광단지,50130,D0,D007,N,N,,06,24시간 이용가능,33.2462149,126.414261
8,중문 관광단지,50130,D0,D007,N,N,,06,24시간 이용가능,33.2462149,126.414261
9,중문 관광단지,50130,D0,D007,N,N,,06,24시간 이용가능,33.2462149,126.414261
14,천지연폭포주차장,50130,B0,B001,N,N,,06,24시간 이용가능,33.2447051,126.5597817
...,...,...,...,...,...,...,...,...,...,...,...
1771,이마트 서귀포점(지상2층주차장),50130,E0,E001,Y,N,,05,10:00~23:00,33.248109,126.507434
1772,이마트 서귀포점(지상2층주차장),50130,E0,E001,Y,N,,05,10:00~23:00,33.248109,126.507434
1773,이마트 서귀포점(지상2층주차장),50130,E0,E001,Y,N,,05,10:00~23:00,33.248109,126.507434
1774,이마트 서귀포점(지상2층주차장),50130,E0,E001,Y,N,,05,10:00~23:00,33.248109,126.507434


In [17]:
df_fast=df_fast.drop_duplicates().reset_index().drop(['index'],axis=1)

In [18]:
df_fast.head(3)

,statNm,zscode,kind,kindDetail,parkingFree,limitYn,limitDetail,chgerType,useTime,lat,lng
0,남원읍 공영주차장,50130,B0,B001,N,N,,06,24시간 이용가능,33.2788767,126.7213139
1,위미의례회관,50130,A0,A004,N,N,,06,24시간 이용가능,33.289615,126.6576261
2,대정감협 인근 공영주차장,50130,B0,B001,N,N,,06,24시간 이용가능,33.2229383,126.2520224


In [19]:
df_final=pd.merge(df_fast,df_fast_count,how='left')

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1067 entries, 0 to 1066
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   statNm       1067 non-null   object
 1   zscode       1067 non-null   object
 2   kind         1067 non-null   object
 3   kindDetail   1067 non-null   object
 4   parkingFree  1067 non-null   object
 5   limitYn      1067 non-null   object
 6   limitDetail  1067 non-null   object
 7   chgerType    1067 non-null   object
 8   useTime      1067 non-null   object
 9   lat          1067 non-null   object
 10  lng          1067 non-null   object
 11  count        1067 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 108.4+ KB


In [21]:
df_final.describe(include='all')

,statNm,zscode,kind,kindDetail,parkingFree,limitYn,limitDetail,chgerType,useTime,lat,lng,count
count,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067.000000
unique,997,3,10,48,2,2,27,4,24,1025,1025,NaN
top,제주월드컵경기장,50110,A0,G004,Y,N,,06,24시간 이용가능,33.4853,126.5092753,NaN
freq,6,670,234,140,922,984,965,654,898,4,4,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.994377
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.983508
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000


In [24]:
df_final.tail()

,statNm,zscode,kind,kindDetail,parkingFree,limitYn,limitDetail,chgerType,useTime,lat,lng,count
1062,제주 방림원,50110,E0,E006,N,N,,06,24시간 이용가능,33.3369377,126.2675195,1
1063,한옥민박 제주,50110,E0,E003,Y,N,,05,24시간 이용가능,33.5471821,126.6931167,1
1064,제주올레돔,50130,E0,E003,Y,N,,05,24시간 이용가능,33.398345396,126.89896843,1
1065,서귀포시산림조합,50130,A0,A004,N,N,,05,24시간 이용가능,33.2606345,126.5129522,1
1066,대정맑은해수사우나,50130,G0,G004,Y,N,,05,24시간 이용가능,33.2197128,126.2485691,1


In [25]:
df_final.to_csv('./charger.csv',index=False)